In [ ]:
!pip install sentencepiece
!pip install nltk
! pip install -U transformers
! pip install -U simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModel

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import operator 

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터셋 읽어오기
data = pd.read_csv('/content/drive/MyDrive/빅데이터처리 개인 폴더/steam_data(to 40000).csv',encoding='UTF-8')
text_data = data['Game Description'].tolist()

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# 1번
def remove_stopwords(sentence):
    # 불용어 제거 
    stopwords = nltk.corpus.stopwords.words('english')
    stop_words = set(stopwords)
    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.casefold() not in stop_words]
    # 다시 문장으로 만들기 
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(r'[^\w\s]', ' ', text)

    return text


def extract_keywords(text, top_k=15):

    text = remove_stopwords(text)
    text = preprocess_text(text)

    print(text)

    # XLNet tokenizer 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
    
    model = AutoModel.from_pretrained("xlnet-base-cased")
    model.to(device)

    # 입력 텍스트 토큰화
    tokens = tokenizer.tokenize(text)

    input_ids = tokenizer.encode(text, return_tensors='pt') # XLNet에 맞게 속성 수정 
    input_ids = input_ids.to(device)


    # XLNet 모델 실행
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        last_hidden_state = outputs.last_hidden_state

    # 키워드 추출
    # last_hidden_state[0].mean(dim=1) : 각 문장의 임베딩 벡터들을 평균화한 하나의 벡터 (해당 문장의 전체 문맥을 나타내는 벡터)
    #  ~.indices:  가장 큰 top_k개의 값들의 인덱스를 나타내는 텐서 (가장 중요한 토큰들의 위치를 확인할 수 있음)
    keyword_ids = torch.topk(last_hidden_state[0].mean(dim=1), top_k).indices.squeeze()
    keywords = [tokens[idx] for idx in set(keyword_ids)]

    return keywords

# 예시 문장
text = text_data[17000]
print('TEXT >>> \n"',text,'"\n\n')

# 키워드 보기
final_key = []

# 추출하기 & 단어만 골라내기
tokens_pos = nltk.pos_tag(set(extract_keywords(text)))

only_nouns = []
for word, pos in tokens_pos:
    if 'NN' in pos:
        only_nouns.append(word)


# 여러 조건들에 맞춰 해당하는 것만 뽑아내기
for w in only_nouns:
  if w == '▁' or w[-1] == 's': continue
  if w[0] == '▁': w = w[1:]
  if len(w) < 3: continue
  lower_w = w.lower()
  if lower_w == 'game': continue
  final_key.append(w.lower())

print('\n\nKEY WORDS : ',set(final_key))


TEXT >>> 
" Find yourself in the middle of the adventures you know so well from the movies of old. Be the reluctant hero during world war 2 who has to save the world. Fight the evil nazi’s and a magical force awoken from her ancient desert slumber. Under a Desert Sun is a unique VR experience with realistic weapon handling and fully realised environmental interaction. Under a Desert Sun will be a full singleplayer campaign game, currently in a "wave shooter state" to test game mechanics. This horde mode is intended to be kept as a seperate game mode, next to full game which among other things will contain:Realistic weapon handling: use your controllers to handle realistic world war 2 weapons. Flight: use your flying skills to fly airplanes and avoid death.Free movement and exploration: explore large environments any way you want.  Impressive VR Graphics: feel immersed in the desert while being overwhelmed by the desert hordes. Incredibly detailed environments, weapons and enemies.VR on

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).




KEY WORDS :  {'weapon', 'war', 'among', 'world', 'sun', 'desert'}


In [ ]:
# 2
def remove_stopwords(sentence):
    # 불용어 제거 
    stopwords = nltk.corpus.stopwords.words('english')
    stop_words = set(stopwords)
    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.casefold() not in stop_words]
    # 다시 문장으로 만들기 
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

def preprocess_text(text):
    # 특수 문자 제거
    text = re.sub(r'[^\w\s]', ' ', text)

    return text


def extract_keywords(text, top_k=15):

    text = remove_stopwords(text)
    text = preprocess_text(text)

    print(text)

    # XLNet tokenizer 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
    
    model = AutoModel.from_pretrained("xlnet-base-cased")
    model.to(device)
    '''
    # 입력 텍스트 토큰화
    tokens = tokenizer.tokenize(text)

    input_ids = tokenizer.encode(text, return_tensors='pt') # XLNet에 맞게 속성 수정 
    input_ids = input_ids.to(device)
    '''
    tokens = tokenizer.tokenize(text)
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # XLNet 모델 실행
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state 
    '''
    # XLNet 모델 실행
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        last_hidden_state = outputs.last_hidden_state
    '''
    # 키워드 추출
    # last_hidden_state[0].mean(dim=1) : 각 문장의 임베딩 벡터들을 평균화한 하나의 벡터 (해당 문장의 전체 문맥을 나타내는 벡터)
    #  ~.indices:  가장 큰 top_k개의 값들의 인덱스를 나타내는 텐서 (가장 중요한 토큰들의 위치를 확인할 수 있음)
    keyword_ids = torch.topk(last_hidden_state[0].mean(dim=1), top_k).indices.squeeze()
    keywords = [tokens[idx] for idx in set(keyword_ids)]

    return keywords

# 예시 문장
text = text_data[17000]
print('TEXT >>> \n"',text,'"\n\n')

# 키워드 보기
final_key = []

# 추출하기 & 단어만 골라내기
tokens_pos = nltk.pos_tag(set(extract_keywords(text)))

only_nouns = []
for word, pos in tokens_pos:
    if 'NN' in pos:
        only_nouns.append(word)


# 여러 조건들에 맞춰 해당하는 것만 뽑아내기
for w in only_nouns:
  if w == '▁' or w[-1] == 's': continue
  if w[0] == '▁': w = w[1:]
  if len(w) < 3: continue
  lower_w = w.lower()
  if lower_w == 'game': continue
  final_key.append(w.lower())

print('\n\nKEY WORDS : ',set(final_key))

In [ ]:
# 3번

# XLNet tokenizer 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = AutoModel.from_pretrained("xlnet-base-cased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def remove_stopwords(sentence):
    # 불용어 제거 
    stopwords = nltk.corpus.stopwords.words('english')
    stop_words = set(stopwords)
    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.casefold() not in stop_words]
    # 다시 문장으로 만들기 
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

# 학습 데이터 전처리
text = text_data[17000]
text = re.sub(r'[^\w\s]', '', text)
text = remove_stopwords(text)
tokens = text.split(' ')

encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)

# XLNet 모델 실행
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = outputs.last_hidden_state

# 각 토큰에 대한 가중치 출력
word_dict = {}
for token, weight in zip(tokens, last_hidden_state[0].mean(dim=1)):
    word_dict[token] = weight.item()

sorted_weight = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)

sorted_weight